In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

# Reading in words
words = open('names.txt', mode='r').read().splitlines()
words[:5]

['emma', 'olivia', 'ava', 'isabella', 'sophia']

In [2]:
# Create mapping from letters to ints:
chars = sorted(list(set(''.join(words)))) # Gets all chars that appear in data
stoi = {s:i+1 for i,s in enumerate(chars)} # Maps each char to a number val
stoi['.'] = 0 # Creates a special character to demarcate the start/end of a word
itos = {i: s for i,s in stoi.items()} # Maps each number to a char
print(itos)

{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '.': 0}


In [3]:
# Now to build the data set
context_size = 3 # Characters taken to predict the next one
X, Y = [], [] # X is characters given to the NN and Y is the Labels (Example: x = emm, y = a)
for w in words:
    context = [0] * context_size
    for char in w + '.':
        ix = stoi[char]
        X.append(context)
        Y.append(ix)
        context = context[1:] + [ix]
X = torch.tensor(X)
Y = torch.tensor(Y)

In [4]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [5]:
# Embedding a lookup table of words for the 'first layer' of our neural network
C = torch.randn(27,2)
emb = C[X]

In [6]:
W1 = torch.randn((6,100)) # Initializing inputs and weight to NN
b1 = torch.randn(100) # Initializes biases as random numbers

# Shape of embeddings is (len(words), 3, 2) so input will be 3*2 = 6
# 100 Neurons was picked arbitrarily
# Next we want to do multiply emb (len(words,3,2)) by W1 (6,100)
# to do this we must concatenate the emb array into 2 dimensions

In [7]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)
parameters = [C, W1, b1, W2, b2]

In [8]:
for p in parameters:
    p.requires_grad = True

In [9]:
# Finding ideal lr
lre = torch.linspace(-3, 0, 1000)
lrs = 10**lre

In [10]:
# counts = logits.exp()
# prob = counts / counts.sum(1, keepdims = True)
# loss = (-prob[torch.arange(prob.shape[0]), Y] - prob.max()).log().mean()

for i in range(1000):
    ix = torch.randint(0, X.shape[0], (32,)) # Creating minibatches

    # Forward pass
    emb = C[X[ix]]
    h = emb.view(-1, 6) @ W1 + b1
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Y[ix]) # Built in, more efficient imlpementation of the above comments (Negative log likelihood)
    # Backwards pass
    for p in parameters:
        p.grad = None
    loss.backward()
    # update
    lr = 10**-2
    for p in parameters:
        p.data += -lr * p.grad
print(loss.item())

3.3161873817443848
